In [1]:
print("ok")

ok


In [2]:
%pwd

'd:\\End-to-end-Medical-Chatbot-Generative-AI\\research'

In [3]:
# import os
# dir_path = os.path.dirname(os.path.realpath(__file__))
# os.chdir(dir_path)
# print(f"{os.getcwd()}")

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'd:\\End-to-end-Medical-Chatbot-Generative-AI'

In [6]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [7]:
#extract data from pdf files
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [8]:
extracted_data = load_pdf_file(data='Data/')

In [9]:
# extracted_data

In [10]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunk = text_splitter.split_documents(extracted_data)
    return text_chunk

In [11]:
text_chunks=text_split(extracted_data)
print("Length of text chunks", len(text_chunks))

Length of text chunks 5860


In [12]:
# text_chunks

In [13]:
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

In [14]:
#download the embeddings from hugging face
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [15]:
embeddings = download_hugging_face_embeddings()

C:\Users\Admin\AppData\Local\Temp\ipykernel_14876\686709670.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
d:\Miniconda\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [17]:
# query_result

In [18]:
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
import os
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GOOGLE_API_KEY=os.environ.get('GOOGLE_API_KEY')


In [42]:
# print(PINECONE_API_KEY)
# print(GOOGLE_API_KEY)

In [21]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [22]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY


In [ ]:
# ##embed each chunk and upsert the embeddings into your Pinecone index

# from langchain_pinecone import PineconeVectorStore

# docsearch = PineconeVectorStore.from_documents(
#     documents=text_chunks,
#     index_name=index_name,
#     embedding=embeddings,
# )
# ####tranh du lieu bi trung lap sau moi lan restart

In [24]:
#load existing index
from langchain_pinecone import PineconeVectorStore
#embed each chunk and upsert the embeddings into Pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [29]:
docsearch

In [30]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})
#3 relevant answer

In [31]:
retrieved_docs=retriever.invoke("What is Acne")

In [32]:
retrieved_docs

[Document(id='1aeffa6f-80b4-45da-892c-28c16bf1ccbb', metadata={'page': 39.0, 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='c2d70524-dd9a-4dce-b5ba-631f1d319de1', metadata={'page': 37.0, 'source': 'Data\\Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is\nthe most common skin disease. It affects nearly 17 million\npeople in the United States. While acne can arise at any'),
 Document(id='e095eeb3-ef1d-4c9e-85ba-7c93a24cf8c9', metadata={'page': 38.0, 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris 

In [33]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="models/gemini-2.5-flash",
    google_api_key= os.getenv("GOOGLE_API_KEY"),
    temperature=0.4,
    max_output_tokens=500,
)

d:\Miniconda\envs\medibot\lib\site-packages\google\api_core\_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [35]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise. "
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [36]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [37]:
# import google.generativeai as genai
# import os

# # Cấu hình API Key
# genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# # Liệt kê các model hỗ trợ phương thức generateContent
# print("Danh sách các models hỗ trợ generateContent:")
# for m in genai.list_models():
#     if 'generateContent' in m.supported_generation_methods:
#         print(f"Name: {m.name}")

In [38]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response["answer"])

Acne is a common skin disorder characterized by pimples on the face, chest, and back. It is the general name given to a condition where the sebaceous glands become inflamed. This occurs when the skin's pores get clogged with oil, dead skin cells, and bacteria.


In [39]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

I'm sorry, but the provided context does not contain information about what "stats" are. It discusses blood counts and athletic heart syndrome.


In [40]:
response = rag_chain.invoke({"input": "What is Antiretroviral drugs?"})
print(response["answer"])

Antiretroviral agents are virustatic drugs that block steps in the replication of a virus. While not curative, their continued use, especially in multi-drug regimens, significantly slows disease progression. These agents are primarily used in HIV (AIDS) therapy.
